## 47. 지연 계산 애트리뷰트가 필요하면 \_\_getattr\_\_, \_\_getattribute\_\_, \_\_setattr\_\_을 사용하라

파이썬에서는 \_\_getattr\_\_이라는 특별 메서드를 사용해 동적 기능을 활용할 수 있다.

어떤 클래스안에 \_\_getattr\_\_ 메서드 정의가 있으면, 이 객체의 인스턴스 딕셔너리에서 찾을 수 없는 애트리뷰트에 접근할 때마다 \_\_getattr\_\_이 호출된다.

In [1]:
class LazyRecord:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = f'{name}의 값'
        setattr(self, name, value)
        return value

In [2]:
data = LazyRecord()
print('이전:', data.__dict__)
print('foo:', data.foo)
print('이후:', data.__dict__)

이전: {'exists': 5}
foo: foo의 값
이후: {'exists': 5, 'foo': 'foo의 값'}


LazyRecord에 로그를 추가해서 언제 호출되는지 확인해보자.

무한 재귀를 피하고 실제 프로퍼티 값을 가져오기 위해 super().\_\_getattr\_\_()을 사용한다.

In [3]:
class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(f'* 호출: __getattr__({name!r}), '
              f'인스턴스 딕셔너리 채워 넣음')
        result = super().__getattr__(name)
        print(f'* 반환: {result!r}')
        return result

In [4]:
data = LoggingLazyRecord()
print('exists: ', data.exists)
print('첫 번째 foo: ', data.foo)
print('두 번째 foo: ', data.foo)

exists:  5
* 호출: __getattr__('foo'), 인스턴스 딕셔너리 채워 넣음
* 반환: 'foo의 값'
첫 번째 foo:  foo의 값
두 번째 foo:  foo의 값


foo 애트리뷰트는 처음에 인스턴스 딕셔너리에 없으므로 맨 처음 foo에 접근하면 \_\_getattr\_\_이 호출된다.

이러한 기능은 지연 계산으로 접근하는 등의 활이 필요할 때 유용하다.

\_\_getattribute\_\_는 객체의 애트리뷰트에 접근 할 때마다 호출된다.

In [6]:
class ValidatingRecord:
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        print(f'* 호출: __getattr__({name!r})')
        try:
            value = super().__getattribute__(name)
            print(f'* {name!r} 찾음, {value!r} 반환')
            return value
        except AttributeError:
            value = f'{name}을 위한 값'
            print(f'* {name!r}를 {value!r}로 설정')
            setattr(self, name, value)
            return value

In [8]:
data = ValidatingRecord()
print('exists: ', data.exists)
print('첫 번째 foo: ', data.foo)
print('두 번째 foo: ', data.foo)

* 호출: __getattr__('exists')
* 'exists' 찾음, 5 반환
exists:  5
* 호출: __getattr__('foo')
* 'foo'를 'foo을 위한 값'로 설정
첫 번째 foo:  foo을 위한 값
* 호출: __getattr__('foo')
* 'foo' 찾음, 'foo을 위한 값' 반환
두 번째 foo:  foo을 위한 값


존재하지 않는 프로퍼티에 동적으로 접근하는 경우에는 AttributeError 예외가 발생한다.

In [9]:
class MissingPropertyRecord:
    def __getattr__(self, name):
        if name == 'bad_name':
            raise AttributeError(f'{name}을 찾을 수 없음')
        return 1 # 무조건 1 반환

In [10]:
data = MissingPropertyRecord()
data.bad_name

AttributeError: bad_name을 찾을 수 없음

파이썬에서 일반적인 기능을 구현하는 코드가 hasattr 내장 함수를 통해 프로퍼티가 존재하는지 검사하는 기능과 getattr 내장 함수를 통해 프로퍼티값을 꺼내오는 기능에 의존할 때도 있다.

이 두 함수도 __getattr__을 호출하기 전에 애트리뷰트 이름을 인스턴스 딕셔너리에서 검색한다.

In [11]:
data = LoggingLazyRecord() # __getattr__을 구현
print('이전: ', data.__dict__)
print('최초에 foo가 있나: ', hasattr(data, 'foo'))
print('이후: ', data.__dict__)
print('다음에 foo가 있나: ', hasattr(data, 'foo'))

이전:  {'exists': 5}
* 호출: __getattr__('foo'), 인스턴스 딕셔너리 채워 넣음
* 반환: 'foo의 값'
최초에 foo가 있나:  True
이후:  {'exists': 5, 'foo': 'foo의 값'}
다음에 foo가 있나:  True


In [12]:
data = ValidatingRecord() # __getattribute__를 구현
print('최초에 foo가 있나: ', hasattr(data, 'foo'))
print('다음에 foo가 있나: ', hasattr(data, 'foo'))

* 호출: __getattr__('foo')
* 'foo'를 'foo을 위한 값'로 설정
최초에 foo가 있나:  True
* 호출: __getattr__('foo')
* 'foo' 찾음, 'foo을 위한 값' 반환
다음에 foo가 있나:  True


__setattr__은 인스턴스의 애트리뷰트에 대입이 이뤄질 때 마다 항상 호출된다.

In [13]:
class SavingRecord:
    def __setattr__(self, name, value):
        # 데이터를 데이터베이스 레코드에 저장한다
        super().__setattr__(name, value)

In [14]:
class LoggingSavingRecord(SavingRecord):
    def __setattr__(self, name, value):
        print(f'* 호출: __setattr__({name!r}, {value!r})')
        super().__setattr__(name, value)

In [15]:
data = LoggingSavingRecord()
print('이전: ', data.__dict__)
data.foo = 5
print('이후: ', data.__dict__)
data.foo = 7
print('최후:', data.__dict__)

이전:  {}
* 호출: __setattr__('foo', 5)
이후:  {'foo': 5}
* 호출: __setattr__('foo', 7)
최후: {'foo': 7}


\_\_getattribute\_\_와 \_\_setattr\_\_의 문제점은 원하든 원하지 않든 객체의 모든 애트리뷰트에 접근할 때마다 호출된다.

예를 들어 어떤 객체와 관련된 딕셔너리에 키가 있을 떄만 이 객체의 애트리뷰트에 접근하고 싶다고 하자.

In [17]:
class BrokenDictionaryRecord:
    def __init__(self, data):
        self._data = {}
        
    def __getattribute__(self, name):
        print(f'* 호출: __getattribute__({name!r})')
        return self._data[name]

In [18]:
data = Brokedata = BrokenDictionaryRecord({'foo': 3})
data.foo

* 호출: __getattribute__('foo')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __getattribute__('_data')
* 호출: __ge

RecursionError: maximum recursion depth exceeded while calling a Python object

이 클래스 정의는 self._data에 대한 접근을 __getattribute__를 통해 수행하도록 요구한다.

하지만 실제로 이 코드를 실행해보면, 파이썬이 스택을 다소모할 때 까지 재귀를 수행하다 죽어버린다.

이 문제는 \_\_attribute\_\_가 self._data에 접근해서 \_\_gettribute\_\_가 다시 호출되기 때문이다.

해결 방법은 super().\_\_attribute\_\_를 호출해 인스턴스 애트리뷰트 딕셔너리에서 값을 가져오는 것이다.

In [19]:
class DictionaryRecord:
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print(f'* 호출: __getattribute__({name!r})')
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

In [20]:
data = DictionaryRecord({'foo': 3})
print('foo: ', data.foo)

* 호출: __getattribute__('foo')
foo:  3


## 기억해야 할 내용
- \_\_getattr\_\_과 \_\_setattr\_\_을 사용해 객체의 애트리뷰트를 지연해 가져오거나 저장할 수 있다.
- \_\_getattr\_\_은 애트리뷰트가 존재하지 않을 때만 호출되지만, \_\_getattribute\_\_는 애트리뷰트를 읽을 때마다 항상 호출된다는 점을 이해하라.
- \_\_getattribute\_\_와 \_\_setattr\_\_에서 무한 재귀를 피하려면 super()에 있는(즉, object 클래스에 있는) 메서드를 사용해 인스턴스 애트리뷰트에 접근하라.